# Libraries

In [10]:
import torch
import os
import shutil
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import random
from glob import glob
import numpy as np
import cv2


# Preparation:


Some global variables

In [11]:
RAW_DIR = "data/raw/140k face"
PROCESSED_DIR = "data/processed"

CATEGORIES = ["fake", "real"]
TRAIN_DIR = os.path.join(PROCESSED_DIR, "train/rgb")
VALID_DIR = os.path.join(PROCESSED_DIR, "valid/rgb")
TEST_DIR = os.path.join(PROCESSED_DIR, "test/rgb")

SHARPEN_TRAIN_DIR = os.path.join(PROCESSED_DIR, "train/sharpen")
SHARPEN_VALID_DIR = os.path.join(PROCESSED_DIR, "valid/sharpen")
SHARPEN_TEST_DIR = os.path.join(PROCESSED_DIR, "test/sharpen")

EDGE_TRAIN_DIR = os.path.join(PROCESSED_DIR, "train/edge")
EDGE_VALID_DIR = os.path.join(PROCESSED_DIR, "valid/edge")
EDGE_TEST_DIR = os.path.join(PROCESSED_DIR, "test/edge")

train_ratio = 0.7
val_ratio = 0.2
random.seed(12347556)

AUG_AMOUNT=0
BATCH_SIZE = 1000

Delete the whole processed_dir structure

In [12]:
if os.path.exists(PROCESSED_DIR):
        shutil.rmtree(PROCESSED_DIR )

# Transformation

The transformation structures

Processing normal images

In [13]:
def process_images(label, size):
    print(f"Working on {label}")
    raw_path = os.path.join(RAW_DIR, label)
    processed_path = PROCESSED_DIR
    os.makedirs(processed_path, exist_ok=True)

    image_paths = [os.path.join(raw_path, f) for f in os.listdir(raw_path) if f.endswith((".png", ".jpg", ".jpeg"))]

    random.shuffle(image_paths)
    
    # Calculate the split index
    train_split_index = int(len(image_paths) * train_ratio)
    val_split_index = int(len(image_paths) * (train_ratio + val_ratio))

    train_paths = image_paths[:train_split_index]
    val_paths = image_paths[train_split_index:val_split_index]
    test_paths = image_paths[val_split_index:]

    create_images(train_paths, "train", size, label)

    create_images(val_paths, "valid", size, label)

    # create_test(test_paths, "test", size, label)
   

In [14]:
def get_unique_filename(base_path, base_name, counter):

    image_name=f"{base_name}_{counter}.jpg"
    
    while True:
        filename = os.path.join(base_path, image_name)
        if not os.path.exists(filename):
            return image_name
        counter += 1

In [15]:
def create_images(images, category, size, label):

    resize_transform = transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
    ])
    
    augmentation_transforms = transforms.Compose([
        transforms.RandomResizedCrop(size, scale=(0.7, 0.9)),
        transforms.RandomApply([
            transforms.ColorJitter(contrast=0.5) 
        ], p=0.5),
        transforms.Resize(size),
        transforms.ToTensor(),
    ])
    save_transform = transforms.ToPILImage()



    ROOT_DIR=os.path.join(PROCESSED_DIR, f"{size[0]}")
    
    if (category=="train"):
        rgb_path=os.path.join(ROOT_DIR, "train/rgb")
        edge_path=os.path.join(ROOT_DIR, "train/edge")
        sharpen_path=os.path.join(ROOT_DIR, "train/sharpen")
    elif(category=="valid"):
        rgb_path=os.path.join(ROOT_DIR, "valid/rgb")
        edge_path=os.path.join(ROOT_DIR, "valid/edge")
        sharpen_path=os.path.join(ROOT_DIR, "valid/sharpen")
    else:
        rgb_path=os.path.join(ROOT_DIR, "test/rgb")
        edge_path=os.path.join(ROOT_DIR, "test/edge")
        sharpen_path=os.path.join(ROOT_DIR, "test/sharpen")

    os.makedirs(rgb_path, exist_ok=True)

    image_counter=0
    for i in tqdm(range(0, len(images), BATCH_SIZE), desc=f"Processing {category}"): #len(images)
        batch = images[i:i + BATCH_SIZE]
        for img_path in batch:

            img = Image.open(img_path).convert("RGB")  # Kép betöltése

            resized_img_name = get_unique_filename(rgb_path, label, image_counter)

                # Save a simple resized version
            resized_img = resize_transform(img)
            make_filters(resized_img, resized_img_name , edge_path, sharpen_path)
            resized_img = save_transform(resized_img)


            resized_img_path=os.path.join(rgb_path, resized_img_name)
                
            resized_img.save(resized_img_path)


                
            image_counter += 1

            if (category=="train"):
                for aug_idx in range(AUG_AMOUNT):
                    aug_img_name= get_unique_filename(rgb_path, label, image_counter)
                    
                    aug_img = augmentation_transforms(img)  # Apply augmentation
                    make_filters(aug_img, aug_img_name , edge_path, sharpen_path)
    
                    
                    aug_img = save_transform(aug_img)  # Convert tensor to PIL image
    
    
                    aug_img_path=os.path.join(rgb_path, aug_img_name)
                        
                    aug_img.save(aug_img_path)
    
                        
                    image_counter += 1

    
    

 move data into respective folder

In [16]:
def make_filters(tensor_image, filename, edge_path, sharpen_path):

    image = tensor_image.permute(1, 2, 0).numpy()

    os.makedirs(edge_path, exist_ok=True)
    os.makedirs(sharpen_path, exist_ok=True)
    
    kernel_edge = np.array([[-1, -1, -1],
                            [-1, 8, -1],
                            [-1, -1, -1]])


    kernel_sharpen = np.array([ [0, -1, 0],
                                [-1, 5, -1],
                                [0, -1, 0]])
    
    # Apply Edge Detection filter
    edge_result = cv2.filter2D(image, -1, kernel_edge)

    # Save the Edge Detection filtered image
    edge_output_path = os.path.join(edge_path, filename)
    cv2.imwrite(edge_output_path, edge_result)

    # Apply Sharpen filter
    sharpen_result = cv2.filter2D(image, -1, kernel_sharpen)

    # Save the Sharpen filtered image
    sharpen_output_path = os.path.join(sharpen_path, filename)
    cv2.imwrite(sharpen_output_path, sharpen_result)

In [17]:
for cat in CATEGORIES:
    process_images(cat, (32,32))

Working on fake


Processing valid: 100%|██████████| 32/32 [06:53<00:00, 12.92s/it]


Working on real


Processing valid: 100%|██████████| 14/14 [02:51<00:00, 12.22s/it]


In [18]:
for cat in CATEGORIES:
    process_images(cat, (64,64))

Working on fake


Processing valid: 100%|██████████| 32/32 [06:11<00:00, 11.62s/it]


Working on real


Processing valid: 100%|██████████| 14/14 [02:38<00:00, 11.35s/it]


In [19]:
for cat in CATEGORIES:
    process_images(cat, (128,128))

Working on fake


Processing valid: 100%|██████████| 32/32 [07:26<00:00, 13.94s/it]


Working on real


Processing valid: 100%|██████████| 14/14 [03:34<00:00, 15.30s/it]
